#### Loading training database

In [91]:
import requests, json, os, warnings, re
import pandas as pd
import datetime
import matplotlib.pyplot as plt
import sys
import glob
data_raw= pd.read_json("./1.TrainingData/all.json") 

In [90]:
data_raw[0:1]


,Exclude,appId,comment,dataSource,date,fee,future,id,label,lemmatized_comment,...,reviewId,reviewer,sentiScore,sentiScore_neg,sentiScore_pos,stemmed,stopwords_removal,stopwords_removal_lemmatization,stopwords_removal_nltk,title
0,NaN,None,"Besides the occasional crash, this is an amazi...",RE2014_app_and_play_store_apps,None,None,0,264,Bug,"besides the occasional crash, this be an amaze...",...,10946,None,3,-1,3,"besid the occa crash, thi is an amaz produc wi...","besides occasional crash, this amazing product...","besides occasional crash, this amaze product w...","besides occasional crash, amazing product tons...",Almost perfect


In [15]:
traindata_noRating=data_raw[data_raw["label"]!="Rating"]
traindata_Rating=data_raw[data_raw["label"]=="Rating"]
traindata_Rating_sample=traindata_Rating.sample(n=1000, random_state=1)

In [16]:
trainData = pd.concat([traindata_noRating, traindata_Rating_sample], axis=0)
trainData.label.value_counts()


Rating            1000
UserExperience     607
Bug                370
Feature            252
Name: label, dtype: int64

#### 1.  STARTING TO TRAIN THE MODEL

In [27]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score

def measuretool(df,classvalue):   
    df["validate"]=df["response"]==df["predict"]
    a_value=sum(df[df["predict"]==classvalue]["validate"])
    ac_value=len(df[df["predict"]==classvalue])
    ab_value=len(df[df["response"]==classvalue])
    Precision_value=a_value/ac_value
    Recall_value=a_value/ab_value
    F1_value=2*Precision_value*Recall_value/(Precision_value+Recall_value)
    return [Precision_value,Recall_value,F1_value]

#### 1.1. BOW + Bigram

In [28]:
# Create training predictors
trainText=trainData.comment
vect = CountVectorizer(ngram_range=(2, 2))
X_train_BOW_Bigram = vect.fit_transform(trainText).toarray()
Y_train=trainData.label

In [29]:
#Training model
classifier_11 = GaussianNB()
classifier_11.fit(X_train_BOW_Bigram,Y_train)

GaussianNB(priors=None, var_smoothing=1e-09)

In [88]:
# Get predict value on the training dataset
Y_train_pred_11 = classifier_11.predict(X_train_BOW_Bigram)
# Accuracy 
accuracy = accuracy_score(Y_train, Y_train_pred_11)
accuracy
# Combince result
df_11 = pd.DataFrame({'response':Y_train, 'predict':Y_train_pred_11})
print(measuretool(df_11,"Bug"))
print(measuretool(df_11,"UserExperience"))
print(measuretool(df_11,"Feature"))
print(measuretool(df_11,"Rating"))

[0.9724517906336089, 0.9540540540540541, 0.9631650750341064]
[1.0, 0.7677100494233937, 0.8685927306616962]
[0.5727272727272728, 1.0, 0.7283236994219653]
[0.88125, 0.846, 0.863265306122449]


#### 1.1 Vectorizing the data

In [45]:
from sklearn.feature_extraction.text import CountVectorizer
vect = CountVectorizer(max_features=1000)
trainingText_stopwords_removal_nlt = vect.fit_transform(trainingText).toarray()
for i in range(5):
    print(sum(trainingText_stopwords_removal_nlt[i]))

5
16
8
14
55


#### 1.2 Training the model

In [51]:
# Naive Bayes 



# Predict Class
y_pred = classifier.predict(trainingText_stopwords_removal_nlt)

# Accuracy 
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(trainingResponse, y_pred)
accuracy

0.25602817664589544

#### 2. STARTING TESTING MODEL

#### 2.1 Loading Testing Data 

In [54]:
Testingdata= pd.read_csv("./2.TestingData/Testing300Sample_completed.csv") 
X_testing=Testingdata["text"]

#### 2.1 Preprocessing the testing data

In [ ]:
import re
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords 


testingText = testingData.text

#Preprocessing the data: Remove none alphabetic characters/Make the word lower case/Remove the stop words

def mutlti_func(x):       
    text= word_tokenize(str(re.sub('[^A-Za-z]', ' ', x)).lower())
    for word in text:
        if word in stopwords.words('english'):
            text.remove(word)
    return text


testingText=testingText.apply(mutlti_func)
testingText.head()

In [ ]:
from nltk.stem.porter import PorterStemmer
from spellchecker import SpellChecker


# Stemming / Spelling 
spell = SpellChecker()
stemmer = PorterStemmer()

def stem_spell_fuc(x):
    for i in range(len(x)):
        #x[i] = stemmer.stem(spell.correction(x[i]))
        x[i] = stemmer.stem(x[i])
    return x

testingText=testingText.apply(stem_spell_fuc)
testingText.head()

In [ ]:
def join_fuc(x):
    x = " ".join(x)
    return x

testingText=testingText.apply(join_fuc)
testingText

#### 2.2 Running testing data

In [ ]:
# Predict Class
y_testing_pred = classifier.predict(testingText)
y_testing_pred.head(20)
